In [88]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
import time
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [76]:
fname = 'data/train.csv'
df = pd.read_csv(fname, sep=';')

In [77]:
f_cat = ['VOIE_DEPOT','COUNTRY','SOURCE_BEGIN_MONTH','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
         'TECHNOLOGIE_SECTOR','TECHNOLOGIE_FIELD','FISRT_INV_COUNTRY','FISRT_INV_TYPE','FIRST_CLASSE',
         'SOURCE_CITED_AGE','SOURCE_IDX_ORI','MAIN_IPC','SOURCE_IDX_RAD']

f_con = list(set(df.columns.values) - set(f_cat) - set(['VARIABLE_CIBLE']))
n_samples, n_features = df.shape[0], len(f_con)

In [78]:
X_train = df[f_con].values
y_train = df['VARIABLE_CIBLE'].values == 'GRANTED'

In [79]:
for i in range(n_samples):
    for j in range(n_features):
        if(isinstance(X_train[i][j], basestring)):
            x = X_train[i][j]
            s = x.split('/')
            X_train[i][j] = int(s[0]) + 12*int(s[1])



In [81]:
imputer = Imputer()
X_train = imputer.fit_transform(X_train)
scale(X_train, copy=False);

In [120]:
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict(X))
n_taken = n_samples
params = {'loss':['hinge'],'alpha':[0.00001],'penalty':['l2'],'n_iter':[5,10,20,30,40],'power_t':[0.5]}
clf = GridSearchCV(SGDClassifier(), params, scorer, n_jobs=2)
start = time.time()
clf.fit(X_train[0:n_taken], y_train[0:n_taken])
print("{} secondes pour fit !".format(time.time() - start))
print("Score de la CV : {}".format(clf.best_score_))
print("Meilleurs pamamètres : {}".format(clf.best_params_))

In [121]:
# 0.607230290732 {'alpha': 0.01, 'loss': 'log'} l2
# 0.604902189663 {'penalty': 'l1', 'alpha': 0.001, 'loss': 'log'}
# 0.606824454887 {'penalty': 'elasticnet', 'alpha': 0.01, 'loss': 'log'}
# 0.607516379177 {'penalty': 'l2', 'alpha': 0.01, 'n_iter': 17, 'loss': 'log', 'power_t':[0.5]}

# 0.547014749323 {'penalty': 'l2', 'alpha': 0.001, 'power_t': 0.5, 'loss': 'perceptron', 'n_iter': 30}
# C'est bien de la grosse merde.

# 0.545011050267 {'penalty': 'l2', 'alpha': 0.001, 'power_t': 0.5, 'loss': 'modified_huber', 'n_iter': 5}
# Not getting any better.

# 0.538653971088 {'penalty': 'l2', 'alpha': 1e-05, 'power_t': 0.5, 'loss': 'hinge', 'n_iter': 20}

#### SGD est définitivement de la merde pour ce problème. On visait un truc du style 0.62 pour rivaliser avec logistic regression avec features polynomiales d'ordre 2. Et random forest. Random forest a bien marché sur ce problème. :D

In [132]:
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
n_taken = n_samples
params = {'min_samples_split': [4], 'oob_score': [True], 'min_weight_fraction_leaf': [0], 'max_depth': [11],
          'min_samples_leaf': [3], 'max_leaf_nodes': [60,70,80,90]}
clf = GridSearchCV(RandomForestClassifier(), params, scorer, n_jobs=2)
start = time.time()
clf.fit(X_train[0:n_taken], y_train[0:n_taken])
print("{} secondes pour fit !".format(time.time() - start))
print("Score de la CV : {}".format(clf.best_score_))
print("Meilleurs pamamètres : {}".format(clf.best_params_))

In [ ]:
# 0.631244960231 {'max_depth': 10}
# 0.629891036885 {'oob_score': True, 'max_leaf_nodes': 80, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'min_samples_split': 4, 'max_depth': 11}
# Autant ne pas tune ce paramètre satanique lulz.

# 0.633689374595 {'min_samples_split': 4, 'oob_score': True, 'min_weight_fraction_leaf': 0, 'max_depth': 11, 'min_samples_leaf': 3}

In [135]:
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
n_taken = n_samples
params = {}
clf = GridSearchCV(AdaBoostClassifier(), params, scorer, n_jobs=2)
start = time.time()
clf.fit(X_train[0:n_taken], y_train[0:n_taken])
print("{} secondes pour fit !".format(time.time() - start))
print("Score de la CV : {}".format(clf.best_score_))
print("Meilleurs pamamètres : {}".format(clf.best_params_))

In [ ]:
# 0.623306132196 default params. Do you even lift ? Mer comment ont-ils fait ?